In [1]:
from MDAnalysis import Universe

topology_file = 'files/3TI6_ose_wt.pdb'
trajectory_file = 'files/ose_wt.dcd'

universe = Universe(topology_file, trajectory_file, trajectory=True, to_guess=("bonds",))
universe.trajectory

C:\Users\ragzo\anaconda3\envs\nanover-dev\Lib\site-packages\MDAnalysis\coordinates\DCD.py:165: DeprecationWarning: DCDReader currently makes independent timesteps by copying self.ts while other readers update self.ts inplace. This behavior will be changed in 3.0 to be the same as other readers. Read more at https://github.com/MDAnalysis/mdanalysis/issues/3889 to learn if this change in behavior might affect you.
  warnings.warn("DCDReader currently makes independent timesteps"


<DCDReader files/ose_wt.dcd with 24 frames of 5877 atoms>

In [2]:
from nanover.mdanalysis import mdanalysis_to_frame_data
from nanover.recording import NanoverRecordingReader
from nanover.recording.reading import MessageEvent
from nanover.recording.playback import MICROSECONDS_TO_SECONDS

def message_events_from_universe(universe: Universe):
    for i, ts in enumerate(universe.trajectory):
        frame = mdanalysis_to_frame_data(universe)
        message = { "frame": frame.pack_to_dict() }

        yield MessageEvent(
            timestamp=.5 * i / MICROSECONDS_TO_SECONDS,
            message=message,
        )

In [3]:
recording_file = "3TI6_ose_wt.nanover.zip"

In [4]:
from nanover.recording.writing import record_messages

with open(recording_file, "wb") as outfile:
    record_messages(outfile, message_events_from_universe(universe))

In [5]:
with NanoverRecordingReader.from_path(recording_file) as reader:
    print(reader)

<NanoverRecordingReader 24 entries spanning 11.5s from '3TI6_ose_wt.nanover.zip'>
